# Vertical Chat
A sample how to build a chat for small business using:

* GPT 35
* Panel
* OpenAI


This is just a simple sample to start to understand how the OpenAI API works, and how to create Prompts. It Is really far from beign a complete solution.
We are going to introduce some interesting points:

* The roles in a conversation.
* How is the conversations’ memory preserved?

Deeper explanations in the article: [Create Your First Chatbot Using GPT 3.5, OpenAI, Python and Panel.](https://medium.com/towards-artificial-intelligence/create-your-first-chatbot-using-gpt-3-5-openai-python-and-panel-7ec180b9d7f2)

In [ ]:
#if you need an API Key from OpenAI
#https://platform.openai.com/account/api-keys

from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [ ]:
client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

def continue_conversation(messages, temperature=0):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=temperature,
    )
    #print(str(response.choices[0].message["content"]))
    return response.choices[0].message.content

In [ ]:
def add_prompts_conversation(_):
    #Get the value introduced by the user
    prompt = client_prompt.value_input
    client_prompt.value = ''

    #Append to the context the User prompt.
    context.append({'role':'user', 'content':f"{prompt}"})

    #Get the response.
    response = continue_conversation(context)

    #Add the response to the context.
    context.append({'role':'assistant', 'content':f"{response}"})

    #Update the panels to show the conversation.
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600)))

    return pn.Column(*panels)

In [ ]:
#Creating the prompt
#read and understand it.
import panel as pn  # GUI

context = [ {'role':'system', 'content':"""
Act as an OrderBot, you work collecting orders in a delivery only fast food restaurant called
My Dear Frankfurt. \
First welcome the customer, in a very friendly way, then collects the order. \
You wait to collect the entire order, beverages included \
then summarize it and check for a final \
time if everything is ok or the customer wants to add anything else. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very friendly style. \
The menu includes \
burger  12.95, 10.00, 7.00 \
frankfurt   10.95, 9.25, 6.50 \
sandwich   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
martra sausage 3.00 \
canadian bacon 3.50 \
romesco sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
vichy catalan 5.00 \
"""} ]

#Creating the panel.
pn.extension()

panels = []

client_prompt = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="talk")

interactive_conversation = pn.bind(add_prompts_conversation, button_conversation)

dashboard = pn.Column(
    client_prompt,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True),
)

dashboard

# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

# **Orderbot No. 1**

In [13]:
import panel as pn
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
import re

# Load environment variables
_ = load_dotenv(find_dotenv())
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# Initialize OpenAI client
client = OpenAI(api_key=OPENAI_API_KEY)

# Define menu prices as a dictionary
menu_prices = {
    "burger": {"large": 12.95, "medium": 10.00, "small": 7.00},
    "frankfurt": {"large": 10.95, "medium": 9.25, "small": 6.50}, 
    "sandwich": {"large": 11.95, "medium": 9.75, "small": 6.75},
    "fries": {"large": 4.50, "small": 3.50},
    "soft drinks": {"large": 3.00, "medium": 2.00, "small": 1.00},
    "water": {"mineral vichy catalan": 5.00, "regular bottle": 3.50},
    "salads": {"caesar": 7.25, "veggie": 6.75},
    "toppings": {
        "extra cheese": 2.00, "mushrooms": 1.50, "martra sausage": 3.00,
        "canadian bacon": 3.50, "romesco sauce": 1.50, "BBQ sauce": 1.00,
        "onion": 1.00, "peppers": 1.00, "fried egg": 2.25,
    },
}

# Initialize context and panels
context = [
    {
        'role': 'system',
        'content': """
        Act as an OrderBot for a fast food restaurant. Clarify sizes, toppings, and extras. 
        Calculate prices strictly based on the menu and summarize the order accurately.
        """
    }
]

panels = []

# Function to calculate the total price based on the context
def calculate_total(order_items):
    total = 0.0
    for item in order_items:
        if item['type'] == 'burger':
            total += menu_prices['burger'].get(item['size'], 0) * item['quantity']
        elif item['type'] == 'frankfurt':
            total += menu_prices['frankfurt'].get(item['size'], 0) * item['quantity']
        elif item['type'] == 'sandwich':
            total += menu_prices['sandwich'].get(item['size'], 0) * item['quantity']
        elif item['type'] == 'fries':
            total += menu_prices['fries'].get(item['size'], 0) * item['quantity']
        elif item['type'] == 'soft drink':
            total += menu_prices['soft drinks'].get(item['size'], 0) * item['quantity']
        elif item['type'] == 'water':
            total += menu_prices['water'].get(item['name'], 0) * item['quantity']
        elif item['type'] == 'salad':
            total += menu_prices['salads'].get(item['name'], 0) * item['quantity']
        elif item['type'] == 'topping':
            total += menu_prices['toppings'].get(item['name'], 0) * item['quantity']
    return total

# Function to extract order items from the model's response
def extract_order_items(response):
    order_items = []
    lines = response.lower().split('\n')
    for line in lines:
        quantity_match = re.search(r'(\d+)\s', line)
        quantity = int(quantity_match.group(1)) if quantity_match else 1

        if "burger" in line:
            size = "large" if "large" in line else "medium" if "medium" in line else "small"
            order_items.append({"type": "burger", "size": size, "quantity": quantity})
        if "frankfurt" in line:
            size = "large" if "large" in line else "medium" if "medium" in line else "small"
            order_items.append({"type": "frankfurt", "size": size, "quantity": quantity})
        if "sandwich" in line:
            size = "large" if "large" in line else "medium" if "medium" in line else "small"
            order_items.append({"type": "sandwich", "size": size, "quantity": quantity})
        if "fries" in line:
            size = "large" if "large" in line else "small"
            order_items.append({"type": "fries", "size": size, "quantity": quantity})
        if "drink" in line or "coke" in line or "sprite" in line:
            size = "large" if "large" in line else "medium" if "medium" in line else "small"
            order_items.append({"type": "soft drink", "size": size, "quantity": quantity})
        for topping in menu_prices['toppings']:
            if topping in line:
                order_items.append({"type": "topping", "name": topping, "quantity": quantity})
    return order_items

# Function to handle the conversation
def add_prompts_conversation(_):
    prompt = client_prompt.value_input
    client_prompt.value = ''

    # Append user's message to the context
    context.append({'role': 'user', 'content': prompt})

    # Call the model for a response
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=context,
        temperature=0
    ).choices[0].message.content

    # Append assistant's response to the context
    context.append({'role': 'assistant', 'content': response})

    # Extract items and calculate total if the response includes order details
    order_items = extract_order_items(response)
    if order_items:
        total_price = calculate_total(order_items)
        response += f"\n\nThe total price for your order is: ${total_price:.2f}"

    # Update the panels
    panels.append(pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(pn.Row('Assistant:', pn.pane.Markdown(response, width=600)))

    return pn.Column(*panels)

# Initialize the Panel dashboard
pn.extension()
client_prompt = pn.widgets.TextInput(value="", placeholder='Enter your order here...')
button_conversation = pn.widgets.Button(name="Submit")
interactive_conversation = pn.bind(add_prompts_conversation, button_conversation)

dashboard = pn.Column(
    client_prompt,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True),
)

dashboard


BokehModel(combine_events=True, render_bundle={'docs_json': {'cf0a1def-8966-4633-bc13-1a0862a6f8d3': {'version…

# **Orderbot No. 2**

In [15]:
import panel as pn
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
import re

# Load environment variables
_ = load_dotenv(find_dotenv())
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# Initialize OpenAI client
client = OpenAI(api_key=OPENAI_API_KEY)

# Define menu prices for an ice cream shop
menu_prices = {
    "scoops": {"single": 3.50, "double": 5.50, "triple": 7.00},
    "cones": {"regular": 1.00, "waffle": 1.50},
    "toppings": {
        "chocolate chips": 0.75, "sprinkles": 0.50, "caramel drizzle": 1.00,
        "whipped cream": 0.75, "cherry": 0.50
    },
}

# Initialize context and panels
context = [
    {
        'role': 'system',
        'content': """
        Act as an OrderBot for a fun ice cream shop called 'Frosty Delights'. 
        Guide the customer through their order, helping them choose scoop sizes, cone types, and toppings.
        Ensure accurate pricing based on the menu and provide a final order summary with the total price.
        """
    }
]

panels = []

# Function to calculate the total price based on the context
def calculate_total(order_items):
    total = 0.0
    for item in order_items:
        if item['type'] == 'scoops':
            total += menu_prices['scoops'].get(item['size'], 0) * item['quantity']
        elif item['type'] == 'cone':
            total += menu_prices['cones'].get(item['name'], 0) * item['quantity']
        elif item['type'] == 'topping':
            total += menu_prices['toppings'].get(item['name'], 0) * item['quantity']
    return total

# Function to extract order items from the model's response
def extract_order_items(response):
    order_items = []
    lines = response.lower().split('\n')
    for line in lines:
        quantity_match = re.search(r'(\d+)\s', line)
        quantity = int(quantity_match.group(1)) if quantity_match else 1

        if "scoop" in line:
            size = "single" if "single" in line else "double" if "double" in line else "triple"
            order_items.append({"type": "scoops", "size": size, "quantity": quantity})
        if "cone" in line:
            name = "waffle" if "waffle" in line else "regular"
            order_items.append({"type": "cone", "name": name, "quantity": quantity})
        for topping in menu_prices['toppings']:
            if topping in line:
                order_items.append({"type": "topping", "name": topping, "quantity": quantity})
    return order_items

# Function to handle the conversation
def add_prompts_conversation(_):
    prompt = client_prompt.value_input
    client_prompt.value = ''

    # Append user's message to the context
    context.append({'role': 'user', 'content': prompt})

    # Call the model for a response
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=context,
        temperature=0
    ).choices[0].message.content

    # Append assistant's response to the context
    context.append({'role': 'assistant', 'content': response})

    # Extract items and calculate total if the response includes order details
    order_items = extract_order_items(response)
    if order_items:
        total_price = calculate_total(order_items)
        response += f"\n\nThe total price for your order is: ${total_price:.2f}"

    # Update the panels
    panels.append(pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(pn.Row('Assistant:', pn.pane.Markdown(response, width=600)))

    return pn.Column(*panels)

# Initialize the Panel dashboard
pn.extension()
client_prompt = pn.widgets.TextInput(value="", placeholder='Enter your ice cream order here...')
button_conversation = pn.widgets.Button(name="Submit")
interactive_conversation = pn.bind(add_prompts_conversation, button_conversation)

dashboard = pn.Column(
    client_prompt,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True),
)

dashboard

BokehModel(combine_events=True, render_bundle={'docs_json': {'0e2cd115-9ef9-449a-98d4-ef066066a076': {'version…

# **Orderbot No. 3**

In [17]:
import panel as pn
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
import re

# Load environment variables
_ = load_dotenv(find_dotenv())
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# Initialize OpenAI client
client = OpenAI(api_key=OPENAI_API_KEY)

# Define menu prices for a Fitness Smoothie Bar
menu_prices = {
    "base": {"banana": 3.50, "strawberry": 4.00, "mango": 4.50},
    "protein": {"whey": 2.00, "vegan": 2.50, "collagen": 3.00},
    "boosts": {
        "chia seeds": 1.00, "flax seeds": 1.00, "peanut butter": 1.50,
        "coconut flakes": 1.25, "matcha": 2.00
    },
}

# Initialize context and panels
context = [
    {
        'role': 'system',
        'content': """
        You are a high-energy, friendly OrderBot for 'BoostBlends', a Fitness Smoothie Bar! 🍓🥭💪
        Guide customers to craft their perfect smoothie. Ask about their base fruit, protein choice,
        and extra boosts. Keep the conversation exciting and engaging. 
        Ensure accurate pricing and always follow up with enthusiasm!
        Confirm the order with the user before providing the final price.
        """
    }
]

panels = []

# Function to calculate the total price based on the context
def calculate_total(order_items):
    total = 0.0
    for item in order_items:
        if item['type'] == 'base':
            total += menu_prices['base'].get(item['name'], 0) * item['quantity']
        elif item['type'] == 'protein':
            total += menu_prices['protein'].get(item['name'], 0) * item['quantity']
        elif item['type'] == 'boost':
            total += menu_prices['boosts'].get(item['name'], 0) * item['quantity']
    return total

# Function to extract order items from the model's response
def extract_order_items(response):
    order_items = []
    lines = response.lower().split('\n')
    for line in lines:
        quantity_match = re.search(r'(\d+)\s', line)
        quantity = int(quantity_match.group(1)) if quantity_match else 1

        for category in menu_prices:
            for item in menu_prices[category]:
                if item in line:
                    order_items.append({"type": category, "name": item, "quantity": quantity})
    return order_items

# Function to handle the conversation
def add_prompts_conversation(_):
    prompt = client_prompt.value_input
    client_prompt.value = ''

    # Append user's message to the context
    context.append({'role': 'user', 'content': prompt})

    # Call the model for a response
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=context,
        temperature=0.7
    ).choices[0].message.content

    # Append assistant's response to the context
    context.append({'role': 'assistant', 'content': response})

    # Extract items and wait for confirmation before calculating the total
    order_items = extract_order_items(response)
    if order_items:
        confirmation_message = "\n\n✅ Here’s your order summary: "
        for item in order_items:
            confirmation_message += f"\n- {item['quantity']}x {item['name'].capitalize()}"
        confirmation_message += "\n\nDoes everything look good? Please confirm before we proceed to payment! 💰"
        response += confirmation_message

    # Update the panels
    panels.append(pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(pn.Row('Assistant:', pn.pane.Markdown(response, width=600, styles={'font-size': '16px', 'font-weight': 'bold', 'color': '#333333'})))

    return pn.Column(*panels, styles={"background-color": "#f7f7f7", "padding": "20px", "border-radius": "10px", "color": "#222222"})

# Initialize the Panel dashboard with improved UI
pn.extension()
client_prompt = pn.widgets.TextInput(value="", placeholder='Type your smoothie order here...')
button_conversation = pn.widgets.Button(name="Blend It! 🥤", button_type="primary")
interactive_conversation = pn.bind(add_prompts_conversation, button_conversation)

dashboard = pn.Column(
    pn.pane.Markdown("## 🚀 Welcome to BoostBlends! Create your ultimate fitness smoothie! 💪🥤", styles={"color": "#2b2b2b", "font-size": "24px", "font-weight": "bold"}),
    client_prompt,
    pn.Row(button_conversation, align="center"),
    pn.panel(interactive_conversation, loading_indicator=True),
    styles={"background-color": "#ffffff", "padding": "30px", "border-radius": "12px", "box-shadow": "0px 4px 10px rgba(0,0,0,0.1)", "color": "#222222"}
)

dashboard

BokehModel(combine_events=True, render_bundle={'docs_json': {'424fb6bc-bebb-4426-975c-3aa32b653069': {'version…

# **Lab Report – Vertial Chat**

# Overview

This project aimed to develop an engaging AI-powered OrderBot across three different scenarios: a fast-food restaurant, an ice cream shop, and a fitness smoothie bar (BoostBlends). Each iteration focused on optimizing conversation flow, pricing accuracy, and user interaction.

Findings & Observations Across All Tryouts

1.	Successful Aspects:

    - Tryout 1 (Fast-Food OrderBot):
        - Successfully handled structured ordering for burgers, fries, and drinks.
        - UI was basic but functional, focusing on price calculation.
        - Encountered pricing inconsistencies due to incorrect quantity tracking.
    - Tryout 2 (Ice Cream Shop - Frosty Delights):
        - Improved order extraction with clear category separation for scoops, cones, and toppings.
        - Added a more interactive UI with a visually appealing style.
        - Faced misinterpretations where GPT occasionally assumed default sizes incorrectly.
    - Tryout 3 (Fitness Smoothie Bar - BoostBlends):
        - Best conversational flow, with enthusiastic tone and structured order confirmation.
        - Implemented double confirmation before calculating the final price.
        - Improved error handling, reducing GPT hallucinations for non-existent items.
        
2.	Challenges & Issues Encountered:
    - Order extraction inconsistencies:
        - Fast-food model struggled with complex item modifications.
        - Ice cream shop bot misclassified some toppings due to vague user input.
    - Pricing errors:
        - Initial models didn’t confirm orders before calculating total price, leading to confusion.
        - Adding a confirmation step in Tryout 3 resolved most of these errors.
    - Flow optimization issues:
        - The first two versions lacked structured follow-ups, leading to abrupt transitions.
        - The smoothie bot successfully implemented friendly follow-ups to guide user choices.

3.  What Didn’t Work Well?
    - Some variations caused GPT to hallucinate toppings or ingredient names not on the menu.
    - The fast-food bot overcomplicated item selection, making it hard to validate complex orders.
    - Early iterations skipped structured confirmations, impacting order accuracy and pricing.

4.  Lessons Learned
    - Confirmation before pricing is crucial: The final version’s two-step verification improved accuracy.
    - Clear category definitions reduce errors: Separating scoops, toppings, and bases made processing easier.
    - UI & conversation flow are key: Structured follow-ups led to smoother interactions and better user engagement.

5.  Future Optimizations
    - Enhance NLP parsing to better handle quantity, size, and add-ons dynamically.
    - Memory module for repeat customers to suggest past favorite orders.
    - Allow order modifications post-confirmation before finalizing the price.
    - Tweak GPT prompt tuning to further reduce hallucinations and ensure menu consistency.
    - Refine user prompts for more intuitive order-building guidance.

6.  Conclusion

    The three iterations provided valuable insights into AI-powered ordering systems. The BoostBlends OrderBot (Tryout 3) emerged as the most optimized, with structured conversation, improved pricing validation, and dynamic follow-ups. Moving forward, these improvements will shape future AI-driven retail and food service ordering experiences.
    🚀🍔🍦🥤 From fast food to fitness smoothies, these AI models have evolved into powerful customer interaction tools!
